In [ ]:
# default_exp exp.i

# Exploration of your data

> This module comprises all the statistical and inference techniques to describe the inner properties of software data. The submodules might include:
>
> - Descriptive statistics
> - Software Metrics
> - Information Theory
> - Learning Principels Detection (Occams' Razor, Biased data, and Data Snooping)
> - Inference: Probabilistic and Causal

In [ ]:
#export
# Imports
import pandas as pd
import sentencepiece as sp
import dit

from collections import Counter
from scipy.stats import sem, t
from numpy import mean
from numpy import std
import statistics as stat
import matplotlib.pyplot as plt


# TODO: Remove when mongo call is implemented
import os

In [ ]:
from nbdev.showdoc import *

In [ ]:
'''
Simulates getting dataframes from a mongodb. To be replaced with actual mango call.
:returns: corpus dataframe 
'''
def simulate_getting_dataframes_from_mongo():
    corpus_data = {'file_name': [], 'data_type': [], 'contents': []}
    path = "./test_data/LibEST_semeru_format/requirements"
    for file in os.listdir(path):
        corpus_data['file_name'].append(file)
        corpus_data['data_type'].append('req')
        with open (os.path.join(path, file), "r") as f:
            corpus_data['contents'].append(f.read())
    path = "./test_data/LibEST_semeru_format/source_code"
    for file in os.listdir(path):
        corpus_data['file_name'].append(file)
        corpus_data['data_type'].append('src')
        with open (os.path.join(path, file), "r") as f:
            corpus_data['contents'].append(f.read())
    path = "./test_data/LibEST_semeru_format/test"
    for file in os.listdir(path):
        corpus_data['file_name'].append(file)
        corpus_data['data_type'].append('test')
        with open (os.path.join(path, file), "r") as f:
            corpus_data['contents'].append(f.read())
    corpus_df = pd.DataFrame(data = corpus_data)
    return corpus_df

In [ ]:
#Hide
#This isn't actually used so we don't need to test it but here you can see how a data frame is organized
#df = simulate_getting_dataframes_from_mongo()
datas = ["Form does not differ from the void, and the void does not differ from the form.", "Form is the void, and the void is form."]
df = pd.DataFrame(data=datas,columns=['contents'] )
df

,contents
0,"Form does not differ from the void, and the vo..."
1,"Form is the void, and the void is form."


In [ ]:
#export
'''
Converts a dataframe into a text file that SentencePiece can use to train a BPE model

:param df: dataframe to be converted
:param output: beginning of filename (output + _text.txt)
:param cols: columns
:returns: dataframe as file
'''
def df_to_txt_file(df, output, cols):
    """"""
    if cols is None: cols = list(df.columns)
    merged_df = pd.concat([df[col] for col in cols])
  
    with open(output + '_text.txt', 'w') as f:
        f.write('\n'.join(list(merged_df)))
    return output + '_text.txt'

In [ ]:
#df_to_txt_file(df,'alexTest', None)

In [ ]:
#ExpTest

#Create a file
df_to_txt_file(df,'output', None)
#Is it there?
test_bool = os.path.isfile('./output_text.txt')
assert(test_bool == True)

os.remove('./output_text.txt')

In [ ]:
#export
'''
Trains a SentencePiece BPE model from a pandas dataframe

:param df: input dataframe
:param output: beginning of filename (output + _text.txt)
:param model_name: name of the model
:param cols: columns
:returns: trained model
'''
def gen_sp_model(df, output, model_name, cols=None):
    
    fname = df_to_txt_file(df, output, cols)
    sp.SentencePieceTrainer.train(f'--input={fname} --model_prefix={output + model_name} --hard_vocab_limit=false --model_type=bpe')
    return output + model_name

In [ ]:
#ExpTest

#Here we test the gen sp model method to see if it properly creates files
actual = gen_sp_model(df,'output','_sp_bpe_modal',cols=["contents"])
expected = os.path.isfile('./' + actual + '.model')
assert(expected)
os.remove('./' + actual + '.model')
expected2 = os.path.isfile('./'+ actual + '.vocab')
assert(expected2)
os.remove('./'+ actual + '.vocab')

os.remove('./output_text.txt')

In [1]:
#export
'''
Encodes text using a pre-trained sp model, returns the occurrences of each token in the text

:param text: input text
:param model_prefix: prefix for trained sp model (f + model_prefix + .model)
:returns: occurrences of each token in text
'''
def encode_text(text, model_prefix):
    sp_processor = sp.SentencePieceProcessor()
    sp_processor.Load(f"{model_prefix}.model")
    token_counts = Counter()
    encoding = sp_processor.encode_as_pieces(text)
    for piece in encoding:
        token_counts[piece] += 1
    return token_counts

In [ ]:
#ExpTest

#This tests the encode text method assuming the sp model(?)

sp_model = gen_sp_model(df,'output','_sp_bpe_modal',cols=["contents"])

actual = encode_text("Form does not differ from the void, and the void does not differ from the form.", 'output_sp_bpe_modal')

expected = Counter({'▁the': 3, '▁does': 2, '▁not': 2, '▁differ': 2, '▁from': 2, '▁void': 2, '▁Form': 1, ',': 1, '▁and': 1, '▁form': 1, '.': 1})

assert(actual == expected)
print(actual)

Counter({'▁the': 3, '▁does': 2, '▁not': 2, '▁differ': 2, '▁from': 2, '▁void': 2, '▁Form': 1, ',': 1, '▁and': 1, '▁form': 1, '.': 1})


In [ ]:
#export
#Notes: token_counts is a counter object.
'''
Takes in a counter object of token occurrences, computes the entropy of the corpus that produced it

:param token_counts: counter object of token occurrences
:returns: shannon entropy of corpus
'''
def dit_shannon(token_counts):
    num_tokens = 0
    for token in token_counts:
        num_tokens += token_counts[token]
    outcomes = list(set(token_counts.elements()))
    frequencies = []
    for token in token_counts:
        frequencies.append((token_counts[token])/num_tokens)
    d = dit.ScalarDistribution(outcomes, frequencies)
    return dit.shannon.entropy(d)

In [ ]:
#ExpTest
#This test takes a sample string and creates a counter that it runs dit shannon on to verify the shannon functionality
sample_txt = "Form does not differ from the void, and the void does not differ from the form."
sample_tcnt = Counter(sample_txt)

answer = dit_shannon(sample_tcnt)
expected = 3.6783847516036627
#assert(answer == expected)
print(answer)
print(expected)
print(sample_tcnt)

3.6783847516036627
3.6783847516036627
Counter({' ': 15, 'o': 10, 'd': 7, 'e': 7, 'f': 7, 'r': 6, 't': 5, 'm': 4, 'i': 4, 'n': 3, 'h': 3, 's': 2, 'v': 2, 'F': 1, ',': 1, 'a': 1, '.': 1})


In [ ]:
#export
'''
Returns a list of the entropies of each entry in a dataframe column

:param df: input dataframe
:param col: column name from dataframe
:param model_prefix: name of pre-trained model
'''
def entropies_of_df_entries(df, col, model_prefix):
    entropies = []
    for data in df[col]:
        token_counts= encode_text(data, model_prefix)
        entropies.append(dit_shannon(token_counts))
    return entropies

In [ ]:
#ExpTest
#Entropies of df test
enty = entropies_of_df_entries(df,'contents','output_sp_bpe_modal')

entropy1 = dit_shannon(Counter({'▁the': 3, '▁does': 2, '▁not': 2, '▁differ': 2, '▁from': 2, 
                                '▁void': 2, '▁Form': 1, ',': 1, '▁and': 1, '▁form': 1, '.': 1}))

entropy2 = dit_shannon(Counter({'▁is': 2, '▁the': 2, '▁void': 2, '▁Form': 1, ',': 1, '▁and': 1, '▁form': 1, '.': 1}))

assert(enty == [entropy1, entropy2])

In [ ]:
# TODO: Finish this such that is finds the entropy of the entire corpus \
#       and preserves the individual token frequencies so that we can   \
#      compute the most common tokens

# def entropy_of_whole_corpus(df, col, model_prefix):
#     '''Returns a dictionary of the entropies of each token in a dataframe corpus'''
#     entropies = {}
#     token_counts = encode_text(pd.concat[col], model_prefix)
#     entropies.append(dit_shannon(token_counts))
#     return entropies

In [ ]:
#FLAG DELETE
# TODO: Do we need this function?
import math
def manual_shannon(token_freqs):
    sum = 0
    for i in token_freqs:
        sum += i * math.log(1/i, 2)
    return sum

In [ ]:
#FLAG DELETE
# TODO: Do we need this function?
def sort_token_data(token_data):
    return sorted(token_data.items() ,  key=lambda x: x[1]["Occurrences"])

# EXPLORATORY ANALYSIS

## LIBest Corpus

In [ ]:
#Hide
# Create a dataframe of the requirements, source code and test case data for LIBest
# Create a sentencepiece model using the entire LIBest corpus
LIB_corpus_df = simulate_getting_dataframes_from_mongo()
LIB_model = gen_sp_model(LIB_corpus_df, output='LIBest', model_name='_sp_bpe_modal', cols=['contents'])